# Excel2DB Playground

Any tools and scripts used in the context of transforming excel data to populate a database.

## Troy200 Dataset transformation for Fonduer

The dataset [TROY200](https://data.mendeley.com/datasets/ydcr7mcrtp/6) is transformed for easier parsing. 2 steps:

* 1) Groundtruth data is in 200 separate files, we transform it into one normalized CSV file with the columns: filename, data, col1, ..., colX, row1, ..., rowY. Where X is the maximum number of column-headers in all the 200 ground truth files, and Y the maximum number of row-headers in all files.

* 2) The data tables are in one single spreadsheet with 200 worksheets. We transform this sheet to save 200 separate spreadsheet files with the name of the worksheet as filename. This is handy to transform them to separate HTML files.


In [ ]:
import os
import pandas as pd

### 1.) Transform Groundtruth Data

In [ ]:
# Ground truth data
gt_path = f"{os.getcwd()}/data/TROY200/original_format/gt"
gt_out_path = f"{os.getcwd()}/data/TROY200"

files = os.listdir(gt_path)
files_xlsx = [f for f in files if f[-4:] == 'xlsx']

def excel_df(f):
    df = pd.read_excel(f"{gt_path}/{f}")
    df['Document'] = f
    return df

# load all files to data frames
gt_dfs = [excel_df(f) for f in files_xlsx]
# concat to a single DF to export as CSV
csv_df = pd.concat(gt_dfs)

csv_df.to_csv(f"{gt_out_path}/troy200_gold.csv", index=False)

### 2.) Transform Data Tables

In [ ]:
from openpyxl import load_workbook
from openpyxl.cell.cell import Cell
from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment

# Data tables per worksheet in the spreadsheet
data_path = f"{os.getcwd()}/data/TROY200/tables-annotated.xlsx"
data_out_path = f"{os.getcwd()}/data/TROY200/tables"

# Style the worksheet by Label
df = pd.read_excel(data_path, sheet_name="Range_Annotations_Data")
styles = {
    "Table": PatternFill("solid", fgColor="FFFFFE"),
    "Data": Font(color="000001"),
    "Header": Font(color="000002"),
    "MetaTitle": Font(color="000003"),
    "Notes": Font(color="000004"),
}

def set_style(ws, cell_range, style):
    rows = ws[cell_range]
    for row in ([rows] if isinstance(rows, Cell) else rows):
        for cell in ([row] if isinstance(row, Cell) else row):
            if (isinstance(style, PatternFill)):
                cell.fill = style
            else:
                cell.font = style


# Iterate worksheets and save as separate xlsx files
wb = load_workbook(filename = data_path)
sheets = wb.sheetnames

for s in sheets:
    wb_copy = load_workbook(filename = data_path)
    df_sheet = df[df["Sheet.Name"] == s]

    # Apply styles
    sheet = wb_copy[s]
    for i, row in df_sheet.iterrows():
        style = styles[row["Annotation.Label"]]
        cell_range = row["Annotation.Range"]
        set_style(sheet, cell_range, style)
    
    # Remove other worksheets
    for s2 in sheets:
        if s2 != s:
            sheet2 = wb_copy[s2]
            wb_copy.remove_sheet(sheet2)
            
    wb_copy.save(f"{data_out_path}/{s}.xlsx")

## Playground

Experimenting